# 手把手教你写 Transformer Decoder Block

在这个教程中，我们将基于你提供的代码，一步步解析并实现一个 Transformer Decoder 模块。

我们将重点关注以下几个核心组件：
1.  **Multi-Head Attention (多头注意力)**：如何拆分头，以及如何计算缩放点积注意力。
2.  **Causal Masking (因果掩码)**：如何确保模型看不到未来的信息。
3.  **Feed-Forward Network (前馈网络)**：两层线性变换加上激活函数。
4.  **Residual Connection & LayerNorm (残差连接与层归一化)**：保持梯度流动和训练稳定性。

---

## 第一步：环境准备与导入

首先，我们需要导入 PyTorch 和必要的数学库。

In [ ]:
import math
import torch
import torch.nn as nn

import warnings
warnings.filterwarnings(action="ignore")

print("环境准备就绪！")

## 第二步：核心注意力计算 (Attention Output)

这是 Transformer 中最复杂的数学部分。我们需要计算 $ \text{softmax}(\\frac{QK^T}{\\sqrt{d}})V $。

在 Decoder 中，我们还需要加上 **Attention Mask**，通常是下三角矩阵，防止模型“作弊”看到未来的 token。

让我们先把这个核心逻辑提取出来理解。

In [ ]:
def attention_output_logic(query, key, value, head_dim, dropout_layer, attention_mask=None):
    # query, key, value 的形状: (batch, nums_head, seq, head_dim)
    
    # 1. 计算相关性 (Attention Scores)
    # 我们需要 Q @ K^T。注意 key 需要转置最后两个维度来匹配矩阵乘法
    key = key.transpose(2, 3)  # 变成 (batch, num_head, head_dim, seq)
    att_weight = torch.matmul(query, key) / math.sqrt(head_dim)

    # 2. 应用 Attention Mask (Causal Masking)
    if attention_mask is not None:
        # 确保 mask 是下三角矩阵 (只看过去)
        attention_mask = attention_mask.tril()
        # 将 mask 为 0 的位置填充为极小的负数 (softmax 后变为 0)
        att_weight = att_weight.masked_fill(attention_mask == 0, float("-1e20"))
    else:
        # 如果没有提供 mask，我们人工构造一个默认的下三角 mask
        attention_mask = torch.ones_like(att_weight).tril()
        att_weight = att_weight.masked_fill(attention_mask == 0, float("-1e20"))

    # 3. Softmax 归一化
    att_weight = torch.softmax(att_weight, dim=-1)
    # (可选) 打印权重用于调试
    # print("Attention Weights sample:", att_weight[0, 0, 0, :]) 

    # 4. Dropout (防止过拟合)
    att_weight = dropout_layer(att_weight)

    # 5. 加权求和 (Weighted Sum)
    mid_output = torch.matmul(att_weight, value)
    # output shape: (batch, nums_head, seq, head_dim)
    
    return mid_output

## 第三步：构建 SimpleDecoder 类

现在我们将所有的组件封装成一个 `nn.Module` 类。

这个类主要包含三个部分：
1.  **`__init__`**: 定义所有层（线性层、Norm、Dropout）。
2.  **`attention_block`**: 处理 Q/K/V 的投影、分头（Reshape）、以及残差连接。
3.  **`ffn_block`**: 前馈神经网络部分。
4.  **`forward`**: 串联整个流程。

### 关于 Normalization (归一化)
你提供的代码使用的是 **Post-Norm** (先相加，再 Norm)：
`Norm(x + sublayer(x))`

*注：现在的 LLaMA 等模型更流行 Pre-Norm (先 Norm，再进入子层)，以及 RMSNorm。但这里我们严格按照你的代码实现 Post-Norm 和 LayerNorm。*

In [ ]:
class SimpleDecoder(nn.Module):
    def __init__(self, hidden_dim, nums_head, dropout=0.1):
        super().__init__()

        self.nums_head = nums_head
        self.head_dim = hidden_dim // nums_head
        self.dropout = dropout

        # --- Layers Definition ---
        
        # 1. Attention 相关的层
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim)
        self.o_proj = nn.Linear(hidden_dim, hidden_dim) # Output projection
        self.drop_att = nn.Dropout(self.dropout)
        self.layernorm_att = nn.LayerNorm(hidden_dim, eps=0.00001)

        # 2. FFN (Feed-Forward Network) 相关的层
        self.up_proj = nn.Linear(hidden_dim, hidden_dim * 4)   # 放大 4 倍
        self.down_proj = nn.Linear(hidden_dim * 4, hidden_dim) # 缩回原维度
        self.act_fn = nn.ReLU() # 激活函数
        self.drop_ffn = nn.Dropout(self.dropout)
        self.layernorm_ffn = nn.LayerNorm(hidden_dim, eps=0.00001)

    # --- 核心逻辑 1: Attention 计算 ---
    def attention_output(self, query, key, value, attention_mask=None):
        # 这里就是我们在第二步中实现的逻辑
        
        # key shape: (batch, num_head, seq, head_dim) -> transpose -> (batch, num_head, head_dim, seq)
        key = key.transpose(2, 3) 
        
        # Scaled Dot-Product
        att_weight = torch.matmul(query, key) / math.sqrt(self.head_dim)

        # Causal Masking
        if attention_mask is not None:
            attention_mask = attention_mask.tril()
            att_weight = att_weight.masked_fill(attention_mask == 0, float("-1e20"))
        else:
            attention_mask = torch.ones_like(att_weight).tril()
            att_weight = att_weight.masked_fill(attention_mask == 0, float("-1e20"))

        att_weight = torch.softmax(att_weight, dim=-1)
        # print("Attention weights max:", att_weight.max().item()) # Debug

        att_weight = self.drop_att(att_weight)
        mid_output = torch.matmul(att_weight, value)
        
        # mid_output: (batch, nums_head, seq, head_dim)
        # 我们需要把它还原回 (batch, seq, hidden_dim)
        
        # transpose(1, 2) 交换 nums_head 和 seq -> (batch, seq, nums_head, head_dim)
        mid_output = mid_output.transpose(1, 2).contiguous()
        
        batch, seq, _, _ = mid_output.size()
        # view 将最后两维合并: nums_head * head_dim = hidden_dim
        mid_output = mid_output.view(batch, seq, -1)
        
        # 最后的线性投影
        output = self.o_proj(mid_output)
        return output

    # --- 模块 1: Attention Block (含 Projection 和 Norm) ---
    def attention_block(self, X, attention_mask=None):
        batch, seq, _ = X.size()
        
        # 1. 投影 (Linear Projections)
        # 2. 分头 (Split Heads): view -> (batch, seq, head, head_dim)
        # 3. 换轴 (Transpose):   transpose -> (batch, head, seq, head_dim)
        # 这样做是为了让 attention 计算时，head 维度独立，seq 维度参与矩阵乘法
        query = self.q_proj(X).view(batch, seq, self.nums_head, -1).transpose(1, 2)
        key = self.k_proj(X).view(batch, seq, self.nums_head, -1).transpose(1, 2)
        value = self.v_proj(X).view(batch, seq, self.nums_head, -1).transpose(1, 2)

        output = self.attention_output(
            query,
            key,
            value,
            attention_mask=attention_mask,
        )
        
        # Post-Norm: Norm(X + Output)
        return self.layernorm_att(X + output)

    # --- 模块 2: Feed Forward Block ---
    def ffn_block(self, X):
        # 1. Up projection (expand)
        up = self.act_fn(self.up_proj(X))
        
        # 2. Down projection (contract)
        down = self.down_proj(up)

        # 3. Dropout
        down = self.drop_ffn(down)

        # 4. Post-Norm: Norm(X + Output)
        return self.layernorm_ffn(X + down)

    # --- 主流程 Forward ---
    def forward(self, X, attention_mask=None):
        # X: (batch, seq, hidden_dim)
        
        # 1. 先过 Attention Block
        att_output = self.attention_block(X, attention_mask=attention_mask)
        
        # 2. 再过 FFN Block
        ffn_output = self.ffn_block(att_output)
        
        return ffn_output

## 第四步：测试运行

最后，我们使用你提供的测试数据来验证模型的输出形状是否正确。

In [ ]:
# 构造输入数据
x = torch.rand(3, 4, 64) # (Batch=3, Seq=4, Hidden=64)
net = SimpleDecoder(64, 8) # Hidden=64, Heads=8

# 构造 Mask
# 原始 Mask: (3, 4)
# 目标 Mask: (Batch, Num_Heads, Seq, Seq) 用于 broadcast
mask = (
    torch.tensor([[1, 1, 1, 1], [1, 1, 0, 0], [1, 1, 1, 0]])
    .unsqueeze(1) # -> (3, 1, 4)
    .unsqueeze(2) # -> (3, 1, 1, 4)
    .repeat(1, 8, 4, 1) # -> (3, 8, 4, 4)
)

output = net(x, mask)
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

# 验证输出形状应与输入一致
assert output.shape == x.shape, "Output shape mismatch!"